<a href="https://colab.research.google.com/github/kaz-yamam0t0/playground/blob/main/TestForSearchingImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# need to manually put target image files in `playground/playground01/images/`
%cd /content/drive/MyDrive
%mkdir -p playground/playground01/images
%cd playground/playground01

In [ ]:
!pip install annoy

In [ ]:
# show one of the images
from IPython.display import Image
Image("images/image_0001.jpg")

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.models import Model

base_model = VGG19(weights="imagenet")
base_model.summary()
#print(base_model.input)
#print(base_model.output)
model = Model(inputs=base_model.input, outputs=base_model.get_layer("fc2").output)
print(model.input)
print(model.output)

In [ ]:
from keras.preprocessing import image
import numpy as np

img_path = "images/image_0001.jpg"
img = image.load_img(img_path, target_size=(224, 224))
input = image.img_to_array(img)
result = model.predict(np.array([input]))
print("result", result)
print("length: ", len(result[0]))

In [ ]:
# training
from annoy import AnnoyIndex
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input

annoy_model = AnnoyIndex(4096)

for i in range (1, 1014):
	img_path = "images/image_%04d.jpg" % i
	img = image.load_img(img_path, target_size=(224, 224))
	x = image.img_to_array(img)
	x = np.expand_dims(x, axis=0)
	x = preprocess_input(x)

	fc2_features = model.predict(x)
	annoy_model.add_item(i, fc2_features[0])

annoy_model.build(1000)
annoy_model.save("image_data.ann")

In [ ]:
# test
from IPython.display import HTML
from base64 import b64encode

from annoy import AnnoyIndex
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input

trained_model = AnnoyIndex(4096)
trained_model.load("image_data.ann")
items = trained_model.get_nns_by_item(14, 3, search_k=-1, include_distances=False)
# print(items)
s = ""
for i in items:
	p = "images/image_%04d.jpg" % i
	f = open(p, 'rb').read()
	data_url = 'data:video/mp4;base64,' + b64encode(f).decode()
	s += f"""
		<h3>{p}</h3>
		<img src="{data_url}">
	"""

HTML(s)